In [1]:
import re
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
from time import sleep
from sklearn.metrics import mean_absolute_error
from huggingface_hub import notebook_login

In [2]:
# Helper function for debugging
def dprint(s, debug):
    if debug:
        print(s)

In [3]:
# --- Check Device ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [4]:
# --- Model Loading ---
# We'll run our models using Hugging Face's transformers library on HPC/ Google Colab/ Lightning.ai
# The Llama models are gated, meaning you must request access on their Hugging Face pages.
# Once you have access, you need to log in here to download the model weights.

# Run this command in your terminal when you are running this notebook for the 1st time
# git config --global credential.helper store

print("Please log in to your Hugging Face account.")
notebook_login()

Please log in to your Hugging Face account.


Token has not been saved to git credential helper.


In [5]:
# This will be our primary model for most of the assignment.
model_id_1 = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
print(f"\nLoading tokenizer for {model_id_1}...")
# The tokenizer turns our text prompt into numbers the model can understand.
tokenizer = AutoTokenizer.from_pretrained(model_id_1)

print(f"Loading model: {model_id_1}...")
# This downloads the model weights to your environment.
# torch_dtype=torch.bfloat16 uses half-precision floats to save memory.
# device_map="auto" automatically puts the model on the GPU if available.
model = AutoModelForCausalLM.from_pretrained(
    model_id_1,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print(f"{model_id_1} model loaded successfully!")


Loading tokenizer for meta-llama/Llama-2-7b-chat-hf...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model: meta-llama/Llama-2-7b-chat-hf...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


meta-llama/Llama-2-7b-chat-hf model loaded successfully!


In [7]:
def call_model(prompt, student_configs, post_processing_fn, model_obj, tokenizer_obj, debug=False):
    """
    Generates a response using the provided local Hugging Face model and tokenizer.
    """
    # 1. Tokenize the input prompt
    inputs = tokenizer_obj(prompt, return_tensors="pt").to(device)

    hf_configs = student_configs.copy()
    if 'max_tokens' in hf_configs:
        # `generate` uses `max_new_tokens` to specify the length of the output
        hf_configs['max_new_tokens'] = hf_configs.pop('max_tokens')
    if 'stop' in hf_configs:
        del hf_configs['stop'] # Stop sequences are handled differently; we'll ignore for simplicity

    # 2. Generate output tokens
    outputs = model_obj.generate(**inputs, **hf_configs).to(device)
    
    # 3. Decode the generated tokens back to a string
    # We slice the output to only get the newly generated text, not the original prompt
    result_new = tokenizer_obj.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    dprint("************ Prompt ************", debug)
    dprint(prompt, debug)
    dprint("\n************ Raw Response ************", debug)
    dprint(result_new, debug)

    # 4. Apply post-processing to extract the final answer
    final_output = post_processing_fn(result_new)
    
    dprint("\n************ Final Output ************", debug)
    dprint(final_output, debug)

    return final_output

In [8]:
def get_addition_pairs(lower_bound, upper_bound, rng):
    """Generates two random integers within a specified range."""
    int_a = int(np.ceil(rng.uniform(lower_bound, upper_bound)))
    int_b = int(np.ceil(rng.uniform(lower_bound, upper_bound)))
    return int_a, int_b

def test_range(added_prompt, prompt_configs, rng, model_obj, tokenizer_obj, n_sample=30,
               lower_bound=1, upper_bound=10, fixed_pairs=None,
               pre_processing=lambda x:x, post_processing=lambda y:y,
               debug=False):
    """
    Tests a language model's addition performance over a range of numbers.

    Args:
        added_prompt (tuple): A tuple containing the prefix and suffix for the prompt.
        prompt_configs (dict): Configuration parameters for the model's generate function.
        rng (numpy.random.Generator): A random number generator instance.
        model_obj (transformers.PreTrainedModel): The loaded Hugging Face model object.
        tokenizer_obj (transformers.PreTrainedTokenizer): The loaded Hugging Face tokenizer object.
        n_sample (int): The number of random pairs to generate if fixed_pairs is None.
        lower_bound (int): The lower bound for number generation.
        upper_bound (int): The upper bound for number generation.
        fixed_pairs (list, optional): A list of specific integer tuples to test.
        pre_processing (function): A function to apply to the input string before prompting.
        post_processing (function): A function to extract the integer answer from the model's output.
        debug (bool): If True, prints detailed debugging information.

    Returns:
        dict: A dictionary containing performance metrics (res, acc, mae, prompt_length).
    """
    # --- Lists for storing results ---
    int_as = []
    int_bs = []
    answers = []
    model_responses = []
    correct = []
    prompts = []
    
    # --- Determine the test cases ---
    iterations = range(n_sample) if fixed_pairs is None else fixed_pairs
    
    for v in tqdm(iterations):
        if fixed_pairs is None:
            # Generate two new numbers if no fixed pairs are provided
            int_a, int_b = get_addition_pairs(lower_bound=lower_bound, upper_bound=upper_bound, rng=rng)
        else:
            # Use the provided fixed pairs
            int_a, int_b = v
            
        # --- Construct the prompt for two numbers ---
        fixed_prompt = f'{int_a}+{int_b}'
        fixed_prompt = pre_processing(fixed_prompt)
        
        prefix, suffix = added_prompt
        prompt = prefix + fixed_prompt + suffix
        
        # --- Get the model's response ---
        model_response = call_model(prompt, prompt_configs, post_processing, model_obj, tokenizer_obj, debug=debug)
        
        # --- Calculate the correct answer for two numbers ---
        answer = int_a + int_b
        
        # --- Append all results for analysis ---
        int_as.append(int_a)
        int_bs.append(int_b)
        prompts.append(prompt)
        answers.append(answer)
        model_responses.append(model_response)
        correct.append((answer == model_response))
        sleep(0.1)

    # --- Create a DataFrame to display the results for two numbers ---
    df = pd.DataFrame({
        'int_a': int_as, 
        'int_b': int_bs, 
        'prompt': prompts, 
        'answer': answers, 
        'response': model_responses, 
        'correct': correct
    })
    print(df)
    
    # --- Calculate and return performance metrics ---
    mae = mean_absolute_error(df['answer'], df['response'])
    acc = df.correct.sum() / len(df)
    prompt_length = len(prefix) + len(suffix)
    res = acc * (1 / prompt_length) * (1 - mae / (1 * 10**4))
    
    return {'res': res, 'acc': acc, 'mae': mae, 'prompt_length': prompt_length}

###  Part 1. Zero Shot Addition

**Example: Zero-shot single-digit addition**

In [25]:
# All of this remains the same
import re

added_prompt = ('Question: What is ', '?\\nAnswer: ')
prompt_config = {'max_tokens': 2,
                'temperature': 0.7,
                'top_k': 50,
                'top_p': 0.6,
                'repetition_penalty': 1,
                'stop': []}

def your_pre_processing(input_string):
    return input_string

def your_post_processing(output_string):
    only_digits = re.sub(r"\D", "", output_string)
    try:
        res = int(only_digits)
    except:
        res = 0
    return res

# The model name string is no longer passed to the function
# It was used in the previous cell to load the 'model' and 'tokenizer' objects
print(f"Testing model: {model_id_1}")
seed = 0
rng = np.random.default_rng(seed)

# This is the only line that changes
res = test_range(
    added_prompt=added_prompt,
    prompt_configs=prompt_config,
    rng=rng,
    model_obj=model, 
    tokenizer_obj=tokenizer,
    n_sample=10,
    lower_bound=1,
    upper_bound=10,
    fixed_pairs=None,
    pre_processing=your_pre_processing,
    post_processing=your_post_processing,
    debug=False
)
print(res)

Testing model: meta-llama/Llama-2-7b-chat-hf


  0%|          | 0/10 [00:00<?, ?it/s]

   int_a  int_b                             prompt  answer  response  correct
0      7      4   Question: What is 7+4?\nAnswer:       11        11     True
1      2      2   Question: What is 2+2?\nAnswer:        4         5    False
2      9     10  Question: What is 9+10?\nAnswer:       19        19     True
3      7      8   Question: What is 7+8?\nAnswer:       15        15     True
4      6     10  Question: What is 6+10?\nAnswer:       16        16     True
5      9      2   Question: What is 9+2?\nAnswer:       11        11     True
6      9      2   Question: What is 9+2?\nAnswer:       11        11     True
7      8      3   Question: What is 8+3?\nAnswer:       11        11     True
8      9      6   Question: What is 9+6?\nAnswer:       15        15     True
9      4      5   Question: What is 4+5?\nAnswer:        9         9     True
{'res': 0.031034172413793103, 'acc': 0.9, 'mae': 0.1, 'prompt_length': 29}


**Example: Zero-shot 7-digit addition**

In [10]:

prompt_config['max_tokens'] = 8
rng = np.random.default_rng(seed)

# The call to test_range is updated to pass the model and tokenizer objects.
res = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model,              # Pass the loaded model object
    tokenizer_obj=tokenizer,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)

print(res)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                        prompt    answer  \
0  6732655  3428081  Question: What is 6732655+3428081?\nAnswer:   10160736   
1  1368762  1148749  Question: What is 1368762+1148749?\nAnswer:    2517511   
2  8319432  9214800  Question: What is 8319432+9214800?\nAnswer:   17534232   
3  6459722  7565469  Question: What is 6459722+7565469?\nAnswer:   14025191   
4  5892625  9415651  Question: What is 5892625+9415651?\nAnswer:   15308276   
5  8342682  1024647  Question: What is 8342682+1024647?\nAnswer:    9367329   
6  8716638  1302271  Question: What is 8716638+1302271?\nAnswer:   10018909   
7  7566899  2580901  Question: What is 7566899+2580901?\nAnswer:   10147800   
8  8768610  5873151  Question: What is 8768610+5873151?\nAnswer:   14641761   
9  3697407  4804185  Question: What is 3697407+4804185?\nAnswer:    8501592   

   response  correct  
0  10155436    False  
1   2517511     True  
2  17530200    False  
3  14027388    False  
4   5892625    

-----------

**Q1a.** In your opinion, what are some factors that cause language model performance to deteriorate from 1 digit to 7 digits?

Answer: In zero-shot addition, performance deteriorates from 1-digit to 7-digits addition because the model optimizes for next-token prediction, not algorithmic computation. For 1 digit addition, the model recalls memorized patterns such as 7 + 4 = 11. However, 7 digits addition requires step-wise reasoning, adding digits right to left, carrying over values. Short addition patterns are common in training dataset, but long 7-digits addition examples are rare. Without examples, the model approximates rather than implement arithmetic logic, and prediction errors keep adding up. Randomness from parameters such as temperature, top-k, or top-p also distort actual results. If the model outputs extra text or spaces, truncation and parsing error can occur during post-processing.

-----------

**Q1b**. Play around with the config parameters ('max_tokens','temperature','top_k','top_p','repetition_penalty')
* What does each parameter represent?
* How does increasing each parameter change the generation?

Answer: Temperature scales the model’s output logits before the softmax operation. A low temperature reduces stochasticity, and ensures identical results for the same input. A higher temperature introduces diversity which is decreases arithmetic precision because it won’t generate definite answer. 
Max Tokens: sets the maximum number of tokens the model can generate in one response. If max token is too small, the model’s output may be truncated. If max token is too large, the model keeps rambling on and produces redundant text even after producing correct result.
Top-P: sampling retains the smallest set of tokens whose cumulative probability exceeds a chosen threshold p and the model samples only from this subset.Higher top-p values include low-probability tokens, increases diversity and error rates in addition outputs. In arithmetic, the correct digit typically has the highest probability, so a small top-p value is necessary. 
Top-K: At each decoding step, top-k restricts sampling to the k most probable tokens. Smaller k focuses the model on the most likely digits, increasing accuracy. Larger k allows random tokens to appear which results in wrong outputs and lower accuracy.
Repetition Penalty: reduces the probability of tokens that have already appeared in the output sequence. Moderate penalty prevents repetition, however, excessive penalty distorts legitimate repeated digits such as 1122111.

Q1c. Do 7-digit addition with Qwen3 8B.

* How does the performance change?
* What are some factors that cause this change?

In [11]:
# --- Before loading Qwen 3, offload Llama 2 to free up VRAM ---

# 1. Delete the model and tokenizer variables from memory.
# Replace 'model' and 'tokenizer' with the actual variable names you used for Llama 2.
del model
del tokenizer

# 2. Run Python's garbage collector and empty PyTorch's CUDA cache.
# This is the crucial step to actually release the GPU memory.
gc.collect()
torch.cuda.empty_cache()

print("Llama 2 model offloaded and GPU memory cleared.")

Llama 2 model offloaded and GPU memory cleared.


In [12]:
# --- Load Qwen 3 8B ---
# This is a different model, so we need to load its specific tokenizer and weights.
model_id_2 = "Qwen/Qwen3-8B"

print(f"\nLoading tokenizer for {model_id_2}...")
tokenizer_2 = AutoTokenizer.from_pretrained(model_id_2)

print(f"Loading model: {model_id_2}...")
model_2 = AutoModelForCausalLM.from_pretrained(
    model_id_2,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print(f"{model_id_2} model loaded successfully!")


Loading tokenizer for Qwen/Qwen3-8B...
Loading model: Qwen/Qwen3-8B...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


Qwen/Qwen3-8B model loaded successfully!


In [13]:
# --- Test on 7-digit addition ---
prompt_config['max_tokens'] = 20

print(prompt_config)
rng = np.random.default_rng(seed)
res_2 = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model_2,              # Pass the loaded model object
    tokenizer_obj=tokenizer_2,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)
print(res_2)

{'max_tokens': 20, 'temperature': 0.7, 'top_k': 50, 'top_p': 0.6, 'repetition_penalty': 1, 'stop': []}


  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                        prompt    answer  \
0  6732655  3428081  Question: What is 6732655+3428081?\nAnswer:   10160736   
1  1368762  1148749  Question: What is 1368762+1148749?\nAnswer:    2517511   
2  8319432  9214800  Question: What is 8319432+9214800?\nAnswer:   17534232   
3  6459722  7565469  Question: What is 6459722+7565469?\nAnswer:   14025191   
4  5892625  9415651  Question: What is 5892625+9415651?\nAnswer:   15308276   
5  8342682  1024647  Question: What is 8342682+1024647?\nAnswer:    9367329   
6  8716638  1302271  Question: What is 8716638+1302271?\nAnswer:   10018909   
7  7566899  2580901  Question: What is 7566899+2580901?\nAnswer:   10147800   
8  8768610  5873151  Question: What is 8768610+5873151?\nAnswer:   14641761   
9  3697407  4804185  Question: What is 3697407+4804185?\nAnswer:    8501592   

         response  correct  
0        10160736     True  
1  25175112517511    False  
2        17534232     True  
3        14025

-----------

Answer: When running zero-shot 7-digits addition, Qwen-3B-8 achieved an accuracy of 0.9 and MAE of 200, indicating that 9 out of 10 predictions were correct and the single incorrect result was almost close to the true sum. It shows higher precision and generalization to 7-digits addition.

Qwen handles multi-digit arithmetic and exhibits lower numerical errors in zero-shot settings. It is trained on larger and diverse datasets containing structured numeric and symbolic text, instruction fine-tuned for factual accuracy. This allows the model to maintain carry propagation more effectively without explicit prompts and examples. Qwen’s decoding is deterministic under same parameter settings temperature, top-p, top-k.

-----------

**Q1d.** Here we're giving our language model the prior that the sum of two 7-digit numbers must have a maximum of 8 digits. (by setting max_token=8). What if we remove this prior by increasing the max_token to 20? 
* Does the model perform well?
* What are some reasons why?

Answer: When max tokens is increased from 8 to 20, the model’s performance degraded, accuracy dropped from 0.9 to 0.6, and MAE increased drastically. The model keeps rambling on and generates longer texts even after giving the correct answer. It appends extra numbers and repeats digits which corrupts post-processing and returns error values. The model didn’t produce numerically valid answers once the output allowed for more tokens.

More tokens increase the number of prediction steps, and small token-level errors keep adding up. Randomness in decoding with temperature, top-p, and top-k generates multiple incorrect digits and distort the actual sum.

In [14]:

prompt_config['max_tokens'] = 20
rng = np.random.default_rng(seed)
res_2 = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model_2,              # Pass the loaded model object
    tokenizer_obj=tokenizer_2,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)
print(res_2)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                        prompt    answer  \
0  6732655  3428081  Question: What is 6732655+3428081?\nAnswer:   10160736   
1  1368762  1148749  Question: What is 1368762+1148749?\nAnswer:    2517511   
2  8319432  9214800  Question: What is 8319432+9214800?\nAnswer:   17534232   
3  6459722  7565469  Question: What is 6459722+7565469?\nAnswer:   14025191   
4  5892625  9415651  Question: What is 5892625+9415651?\nAnswer:   15308276   
5  8342682  1024647  Question: What is 8342682+1024647?\nAnswer:    9367329   
6  8716638  1302271  Question: What is 8716638+1302271?\nAnswer:   10018909   
7  7566899  2580901  Question: What is 7566899+2580901?\nAnswer:   10147800   
8  8768610  5873151  Question: What is 8768610+5873151?\nAnswer:   14641761   
9  3697407  4804185  Question: What is 3697407+4804185?\nAnswer:    8501592   

         response  correct  
0        10160736     True  
1         2517511     True  
2        17534232     True  
3  14025191769

In [15]:
# 1. Delete the model and tokenizer variables from memory.
# Replace 'model' and 'tokenizer' with the actual variable names you used for Llama 2.
del model_2
del tokenizer_2

# 2. Run Python's garbage collector and empty PyTorch's CUDA cache.
# This is the crucial step to actually release the GPU memory.
gc.collect()
torch.cuda.empty_cache()

print(f"{model_id_2} offloaded and GPU memory cleared.")

Qwen/Qwen3-8B offloaded and GPU memory cleared.


### Part 2. In Context Learning

We will try to improve the performance of 7-digit addition via in-context learning.
We will use [llama-2-7b]. Below is a simple example.

In [16]:
print(f"\nLoading tokenizer for {model_id_1}...")
# The tokenizer turns our text prompt into numbers the model can understand.
tokenizer = AutoTokenizer.from_pretrained(model_id_1)

print(f"Loading model: {model_id_1}...")
# This downloads the model weights to your environment.
# torch_dtype=torch.bfloat16 uses half-precision floats to save memory.
# device_map="auto" automatically puts the model on the GPU if available.
model = AutoModelForCausalLM.from_pretrained(
    model_id_1,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print(f"{model_id_1} model loaded successfully!")


Loading tokenizer for meta-llama/Llama-2-7b-chat-hf...
Loading model: meta-llama/Llama-2-7b-chat-hf...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

meta-llama/Llama-2-7b-chat-hf model loaded successfully!


In [21]:

added_prompt = ('Question: What is 3+7?\nAnswer: 10\n Question: What is ', '?\nAnswer: ') # Question: What is a+b?\nAnswer:
prompt_config = {'max_tokens': 8,
                'temperature': 0.7,
                'top_k': 50,
                'top_p': 0.6,
                'repetition_penalty': 1,
                'stop': []}
rng = np.random.default_rng(seed)
res = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model,              # Pass the loaded model object
    tokenizer_obj=tokenizer,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)

print(res)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                             prompt  \
0  6732655  3428081  Question: What is 3+7?\nAnswer: 10\n Question:...   
1  1368762  1148749  Question: What is 3+7?\nAnswer: 10\n Question:...   
2  8319432  9214800  Question: What is 3+7?\nAnswer: 10\n Question:...   
3  6459722  7565469  Question: What is 3+7?\nAnswer: 10\n Question:...   
4  5892625  9415651  Question: What is 3+7?\nAnswer: 10\n Question:...   
5  8342682  1024647  Question: What is 3+7?\nAnswer: 10\n Question:...   
6  8716638  1302271  Question: What is 3+7?\nAnswer: 10\n Question:...   
7  7566899  2580901  Question: What is 3+7?\nAnswer: 10\n Question:...   
8  8768610  5873151  Question: What is 3+7?\nAnswer: 10\n Question:...   
9  3697407  4804185  Question: What is 3+7?\nAnswer: 10\n Question:...   

     answer  response  correct  
0  10160736  10154439    False  
1   2517511   2517511     True  
2  17534232  17533280    False  
3  14025191   7322295    False  
4  15308276  1030816

**Q2a**.
* How does the performance change with the baseline in-context learning prompt? (compare with "Example: Zero-shot 7-digit addition" in Q1)
* What are some factors that cause this change?

Answer: With the baseline in-context learning prompt, accuracy dropped to 10% and mae was approx 9.84 million. Compared to zero-shot 7-digit additions with maximum 8 tokens with the same decoding parameters, in-context learning performances drastically decreases.

The single example is a 1-digit addition, which has no carries and doesn’t match with the 7 digit addition. The model learns the format Question-Answer but doesn’t learn the algorithmic reasoning for 7-digits addition. Limit of 8 tokens result in incomplete and truncated answer if the model produces extra token such as space, commas, or newline. The parameters values introduces higher stochasticity and no stop sequence makes the model to keep generating unnecessary tokens. Also, 1-digit addition example teaches the model that answers might be short, single-digit numbers.

-----------

Now we will remove the prior on output length and re-evaluate the performance of our baseline one-shot learning prompt. We need to modify our post processing function to extract the answer from the output sequence. In this case, it is the number in the first line that starts with "Answer: ".

**Q2b**.
* Modify the post processing function
* How does the performance change when we relax the output length constraint? (compare with Q2a)
* What are some factors that cause this change?

Answer: The post-processing function extracts the final answer from the model’s output. It splits the output at the Answer:, isolates the first line, cleans by removing commas, underscores, and extra spaces. Identifies and extracts numeric patterns, first look 8-digit numbers and then 7-digit numbers. If an 8-digit number is found, the last occurrence is returned, or else the function checks for a 7-digit number.

Accuracy dropped from 10% to 0%, mae decreased from 9,844,393 to 5,304,260 when max tokens was increased.

With higher max tokens, the model produces explanations, intermediate sums, commas/spaces, multiple numbers, adds extra texts and possibly prints answer in next line. This breaks the modified post processing function, and returns wrong values. This reduces accuracy and increases mae.

In [22]:
#Write your updated post processing function here
def your_post_processing(output_string: str) -> int:
    after = output_string.split("Answer:", 1)[-1]
    first_line = after.splitlines()[0] if after else ""
    cleaned = first_line.replace(",", "").replace("_", " ").strip()

    def _safe_int(s):
        try:
            return int(s)
        except Exception:
            return 0

    m8 = re.findall(r"(?<!\d)(\d{8})(?!\d)", cleaned)
    if m8:
        return _safe_int(m8[-1]) 
    m7 = re.findall(r"(?<!\d)(\d{7})(?!\d)", cleaned)
    if m7:
        return _safe_int(m7[-1]) 
    return 0

In [23]:
prompt_config['max_tokens'] = 50 # changed from 8, assuming we don't know the output length
                
rng = np.random.default_rng(seed)
res = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model,              # Pass the loaded model object
    tokenizer_obj=tokenizer,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)
print(res)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                             prompt  \
0  6732655  3428081  Question: What is 3+7?\nAnswer: 10\n Question:...   
1  1368762  1148749  Question: What is 3+7?\nAnswer: 10\n Question:...   
2  8319432  9214800  Question: What is 3+7?\nAnswer: 10\n Question:...   
3  6459722  7565469  Question: What is 3+7?\nAnswer: 10\n Question:...   
4  5892625  9415651  Question: What is 3+7?\nAnswer: 10\n Question:...   
5  8342682  1024647  Question: What is 3+7?\nAnswer: 10\n Question:...   
6  8716638  1302271  Question: What is 3+7?\nAnswer: 10\n Question:...   
7  7566899  2580901  Question: What is 3+7?\nAnswer: 10\n Question:...   
8  8768610  5873151  Question: What is 3+7?\nAnswer: 10\n Question:...   
9  3697407  4804185  Question: What is 3+7?\nAnswer: 10\n Question:...   

     answer  response  correct  
0  10160736         0    False  
1   2517511         0    False  
2  17534232         0    False  
3  14025191   7322188    False  
4  15308276         

-----------

**Q2c.** Let's change our one-shot learning example to something more "in-distribution". Previously we were using 1-digit addition as an example. Let's change it to 7-digit addition (1234567+1234567=2469134). 
* Evaluate the performance with max_tokens = 8.
* Evaluate the performance with max_tokens = 50.
* How does the performance change from 1-digit example to 7-digit example?

Answer: With the in-distribution one-shot prompt and 7-digit addition, the model’s performance improves compared to the 1-digit addition as example.
Results
– res: -0.204
– acc: 0.2 or 20%
– mae: 816,560.1
– prompt length: 79
7-digit addition example makes the in-distribution one-shot prompt allows the model to learn the addition pattern much better, it knows that output should have seven or eight digits. The accuracy doubled to 20%, and mae decreased drastically from 9.84 million to 0.82 million. It shows that the predictions are numerically closer to the actual results. The results show that in-distribution one-shot prompt improves performance even with few context length.

With the in-distribution one-shot prompt and max tokens 50, the model’s performance decreases drastically compared to max tokens 8.
Results
– res: −0.0
– acc: 0.0 or 0%
– mae: 3.26 × 1031
– prompt length: 79
With max tokens 50 increased from 8 to 50 tokens, the model continues generating additional tokens beyond the correct numeric output instead of clean numerical answers which are mostly concatenations of multiple numbers rather than a valid sum. Local token prediction noise cascades under stochastic decoding. It causes mae to increase drastically and the accuracy to drop to zero.

When the one-shot example was changed from a 1-digit addition to a 7-digit addition, the model’s performance improved under the same parameter values max tokens 8. Accuracy increased from 10% to 20%, and mae dropped drastically from about 9.84 million to roughly 0.82 million, and the predictions became numerically closer to the actual sum. This happened because the new example matched the distribution of the inputs, allows the model learn 7 digits addition much better. However, with max tokens 50, the model’s performed poorly. It increased mae to an extreme value to 3.26 × 1031 and reduced accuracy to zero.

In [26]:

prompt_config['max_tokens'] = 8 
added_prompt = ('Question: What is 1234567+1234567?\nAnswer: 2469134\nQuestion: What is ', '?\nAnswer: ') # Question: What is a+b?\nAnswer:
res = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model,              # Pass the loaded model object
    tokenizer_obj=tokenizer,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)
print(res)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                             prompt  \
0  1254878  2118550  Question: What is 1234567+1234567?\nAnswer: 24...   
1  7035620  6824705  Question: What is 1234567+1234567?\nAnswer: 24...   
2  6538466  4453098  Question: What is 1234567+1234567?\nAnswer: 24...   
3  9974889  9827518  Question: What is 1234567+1234567?\nAnswer: 24...   
4  7169878  6854133  Question: What is 1234567+1234567?\nAnswer: 24...   
5  7196020  4500293  Question: What is 1234567+1234567?\nAnswer: 24...   
6  2215869  7493395  Question: What is 1234567+1234567?\nAnswer: 24...   
7  5728189  3792177  Question: What is 1234567+1234567?\nAnswer: 24...   
8  5372518  9005390  Question: What is 1234567+1234567?\nAnswer: 24...   
9  9406391  4220157  Question: What is 1234567+1234567?\nAnswer: 24...   

     answer  response  correct  
0   3373428   3373428     True  
1  13860325  13859925    False  
2  10991564  10983464    False  
3  19802407  19802497    False  
4  14024011  1396400

In [27]:

prompt_config['max_tokens'] = 50 
res = test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    model_obj=model,              # Pass the loaded model object
    tokenizer_obj=tokenizer,      # Pass the loaded tokenizer object
    n_sample=10, 
    lower_bound=1000000, 
    upper_bound=9999999, 
    fixed_pairs=None, 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    debug=False
)
print(res)

  0%|          | 0/10 [00:00<?, ?it/s]

     int_a    int_b                                             prompt  \
0  6143768  3896825  Question: What is 1234567+1234567?\nAnswer: 24...   
1  6348700  4041201  Question: What is 1234567+1234567?\nAnswer: 24...   
2  4524571  9012469  Question: What is 1234567+1234567?\nAnswer: 24...   
3  3044419  6608684  Question: What is 1234567+1234567?\nAnswer: 24...   
4  1756139  8493797  Question: What is 1234567+1234567?\nAnswer: 24...   
5  8083884  3154325  Question: What is 1234567+1234567?\nAnswer: 24...   
6  8888358  1527113  Question: What is 1234567+1234567?\nAnswer: 24...   
7  4025054  2352516  Question: What is 1234567+1234567?\nAnswer: 24...   
8  5053054  8166918  Question: What is 1234567+1234567?\nAnswer: 24...   
9  3075780  1468192  Question: What is 1234567+1234567?\nAnswer: 24...   

     answer                          response  correct  
0  10040593  10034513987654323456781221812175    False  
1  10389901  10389301987654376543217320973456    False  
2  13537040  5

-----------

**Q2d.** Let's look at a specific example with large absolute error. 
* Run the cell at least 5 times. Does the error change each time? Why?
* Can you think of a prompt to reduce the error?
* Why do you think it would work?
* Does it work in practice? Why or why not?

Answer: With each run, the error changes drastically, mean absolute error ranges from (1.01 × 1031) to (1.01 × 1032). This happens even though the prompt and inputs are identical. The randomness arises from sampling by temperature, top k, and top p. The model does not produce deterministic outputs and the responses vary widely. The model produces long concatenated sequences, increases the integer values, and produces large mae values. The error fluctuates because small differences in the model’s random sampling causes different numeric outputs once token generation begins, and the post-processing interprets those long texts as enormous numbers.

You’re a calculator. Answer the question with final result only. Result should be in digits only, no spaces, commas, or text. Don’t give explanation of the answer. Don’t echo the question. Follow these rules strictly: Add digits from right to left. If a carry is 10 or more, add it to the next digit. If there is only a carry left, prepend it.

The prompt works because it constrains both the model’s behavior and the decoding process. By instructing the model to only write the final number and not add extra text or questions, it doesn’t allow the model to keep generating text after the answer. And it makes the post processing eaier to extract the correct number and parse it.

In practice, the prompt reduces large errors and answer will be off by only one or two digits. However, it does not guarantee perfect accuracy for multi-digit addition because the model is performing pattern completion, not algorithmic computation. While the constraints improve format consistency and fewer text generation, the model can still miscalculate due to limited numerical reasoning capability. It is trained in-distribution only on seven digits addition. Thus, the prompt makes formatting better and decreases large mae values, but it doesn’t solve the numeric reasoning limitations of the model.

In [32]:
test_range(
    added_prompt=added_prompt, 
    prompt_configs=prompt_config, 
    rng=rng, 
    fixed_pairs=[(9090909,1010101)], 
    pre_processing=your_pre_processing, 
    post_processing=your_post_processing, 
    model_obj=model, 
    tokenizer_obj=tokenizer, 
    debug=True
)

  0%|          | 0/1 [00:00<?, ?it/s]

************ Prompt ************
Question: What is 1234567+1234567?
Answer: 2469134
Question: What is 9090909+1010101?
Answer: 

************ Raw Response ************
101010091
Question: What is 12345678+12345678?
Answer: 24691356
Question: What is

************ Final Output ************
101010091123456781234567824691356
     int_a    int_b                                             prompt  \
0  9090909  1010101  Question: What is 1234567+1234567?\nAnswer: 24...   

     answer                           response  correct  
0  10101010  101010091123456781234567824691356    False  


{'res': -0.0, 'acc': 0.0, 'mae': 1.0101009112345678e+32, 'prompt_length': 79}